# IMDb Knowledge Graph Builder for TransE Training

This notebook builds a Knowledge Graph from IMDb TSV files stored in Google Drive.

**Modes:**
- **Full KG**: Processes all movies (produces ~81M triples, ~16.5M entities)
- **Filtered KG**: Filters movies by year, type, and top billing (target: 5-10M triples)

**Entities:**
- Movies (tconst)
- Persons (nconst)
- Genres (from title.basics genres column)

**Relations:**
- (movie) --HAS_GENRE--> (genre)
- (person) --DIRECTED--> (movie)
- (person) --WROTE--> (movie)
- (person) --ACTED_IN--> (movie)

In [ ]:
# Install required packages (if not already installed)
!pip install pandas tqdm -q

In [ ]:
# Configuration
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import os
import random

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# ============================================================================
# PIPELINE MODE SELECTION
# ============================================================================
USE_FILTERED_MODE = True  # Set to False for full KG, True for filtered KG

# ============================================================================
# FILTERING CONFIGURATION (only used if USE_FILTERED_MODE = True)
# ============================================================================
YEAR_MIN = 1970  # Keep only movies with startYear >= YEAR_MIN
TOP_BILLING = 5  # Keep only actors/actresses with ordering <= TOP_BILLING

# ============================================================================
# DRY-RUN MODE (for quick testing)
# ============================================================================
LIMIT_ROWS = None  # Set to integer (e.g., 10000) to limit rows per file for testing
LIMIT_MOVIES = None  # Alternative: limit number of movies to process (after filtering)

# ============================================================================
# FULL KG STATISTICS (for comparison - update if you know the full KG stats)
# ============================================================================
FULL_KG_STATS = {
    'movies': 16500000,  # Approximate - update with actual if known
    'triples': 81000000,  # Approximate - update with actual if known
    'entities': 16500000  # Approximate - update with actual if known
}

# ============================================================================
# PATHS AND FILES
# ============================================================================
DRIVE_ROOT = '/content/drive/MyDrive'
INPUT_DIR = f'{DRIVE_ROOT}/Knowledge Graph'  # Folder containing TSV files

# Output directory based on mode
if USE_FILTERED_MODE:
    OUTPUT_DIR = f'{DRIVE_ROOT}/kg_output_filtered'
else:
    OUTPUT_DIR = f'{DRIVE_ROOT}/kg_output'

# File names
FILES = {
    'basics': 'title.basics.tsv',
    'crew': 'title.crew.tsv',
    'principals': 'title.principals.tsv',
    'ratings': 'title.ratings.tsv'  # Not used for triples but can be read
}

print("=" * 60)
print("CONFIGURATION")
print("=" * 60)
print(f"Mode: {'FILTERED KG' if USE_FILTERED_MODE else 'FULL KG'}")
print(f"Input directory: {INPUT_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
if USE_FILTERED_MODE:
    print(f"  Year minimum: {YEAR_MIN}")
    print(f"  Top billing limit: {TOP_BILLING}")
print(f"Dry-run limit rows: {LIMIT_ROWS if LIMIT_ROWS else 'None'}")
print(f"Dry-run limit movies: {LIMIT_MOVIES if LIMIT_MOVIES else 'None'}")
print("=" * 60)


CONFIGURATION
Mode: FILTERED KG
Input directory: /content/drive/MyDrive/Knowledge Graph
Output directory: /content/drive/MyDrive/kg_output_filtered
  Year minimum: 1970
  Top billing limit: 5
Dry-run limit rows: None
Dry-run limit movies: None


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory ready: {OUTPUT_DIR}")


Mounted at /content/drive
Output directory ready: /content/drive/MyDrive/kg_output_filtered


In [ ]:
# Helper functions

def safe_split(value, sep=','):
    """Safely split a value, handling missing values marked as \\N"""
    if pd.isna(value) or value == '\\N' or value == '':
        return []
    return [v.strip() for v in str(value).split(sep) if v.strip() and v.strip() != '\\N']

def read_tsv_chunked(filepath, limit=None, chunksize=100000):
    """Read TSV file with optional row limit and chunking"""
    if limit:
        # For dry-run, read only first N rows
        df = pd.read_csv(filepath, sep='\t', nrows=limit, low_memory=False)
        return [df]  # Return as single chunk
    else:
        # Full processing with chunking
        return pd.read_csv(filepath, sep='\t', chunksize=chunksize, low_memory=False)

def get_file_path(filename):
    """Get full path for a file in INPUT_DIR"""
    return os.path.join(INPUT_DIR, filename)

print("Helper functions defined")


Helper functions defined


In [ ]:
# Step 1: Extract entities and build triples from title.basics.tsv
# Entities: movies (tconst), genres
# Relations: (movie) --HAS_GENRE--> (genre)
# Apply filtering if USE_FILTERED_MODE is True

print("=" * 60)
print("Step 1: Processing title.basics.tsv")
print("=" * 60)

triples = []
movies = set()
genres = set()
kept_movies = set()  # Set of tconst that passed all filters (used in later steps)

basics_path = get_file_path(FILES['basics'])
print(f"Reading: {basics_path}")
if USE_FILTERED_MODE:
    print(f"Filtering: titleType='movie', startYear>={YEAR_MIN}, isAdult=0, genres not missing")

chunks = read_tsv_chunked(basics_path, limit=LIMIT_ROWS)
movies_processed = 0
movies_filtered_out = 0

for chunk_idx, chunk in enumerate(chunks):
    print(f"Processing chunk {chunk_idx + 1}... (rows: {len(chunk)})")

    # Filter out rows with missing tconst
    chunk = chunk[chunk['tconst'].notna() & (chunk['tconst'] != '\\N')]

    if USE_FILTERED_MODE:
        # Apply filtering rules
        # 1. Keep only titleType == "movie"
        chunk = chunk[chunk['titleType'] == 'movie']

        # 2. Exclude isAdult == 1
        chunk = chunk[(chunk['isAdult'].isna()) | (chunk['isAdult'] == 0) | (chunk['isAdult'] == '0')]

        # 3. Filter by startYear >= YEAR_MIN and not missing
        chunk = chunk[chunk['startYear'].notna() & (chunk['startYear'] != '\\N')]
        # Convert startYear to numeric, handling errors
        chunk['startYear_numeric'] = pd.to_numeric(chunk['startYear'], errors='coerce')
        chunk = chunk[chunk['startYear_numeric'] >= YEAR_MIN]

        # 4. Drop rows with missing genres
        chunk = chunk[chunk['genres'].notna() & (chunk['genres'] != '\\N') & (chunk['genres'] != '')]

    movies_processed += len(chunk)

    for _, row in tqdm(chunk.iterrows(), total=len(chunk), desc=f"Chunk {chunk_idx + 1}"):
        movie_id = str(row['tconst']).strip()
        if not movie_id or movie_id == '\\N':
            continue

        # Check LIMIT_MOVIES if set
        if LIMIT_MOVIES and len(kept_movies) >= LIMIT_MOVIES:
            break

        movies.add(movie_id)
        kept_movies.add(movie_id)

        # Extract genres
        genre_list = safe_split(row.get('genres', ''))
        for genre in genre_list:
            if genre:
                genres.add(genre)
                triples.append(('HAS_GENRE', movie_id, genre))

    if LIMIT_MOVIES and len(kept_movies) >= LIMIT_MOVIES:
        print(f"Reached LIMIT_MOVIES={LIMIT_MOVIES}, stopping processing")
        break

movies_filtered_out = movies_processed - len(kept_movies)

print(f"\nStep 1 Complete:")
print(f"  Movies processed: {movies_processed:,}")
if USE_FILTERED_MODE:
    print(f"  Movies kept after filtering: {len(kept_movies):,}")
    print(f"  Movies filtered out: {movies_filtered_out:,}")
else:
    print(f"  Movies found: {len(movies):,}")
print(f"  Genres found: {len(genres)}")
print(f"  HAS_GENRE triples: {sum(1 for t in triples if t[0] == 'HAS_GENRE'):,}")


Step 1: Processing title.basics.tsv
Reading: /content/drive/MyDrive/Knowledge Graph/title.basics.tsv
Filtering: titleType='movie', startYear>=1970, isAdult=0, genres not missing
Processing chunk 1... (rows: 100000)


Chunk 1: 100%|██████████| 22597/22597 [00:01<00:00, 21894.55it/s]


Processing chunk 2... (rows: 100000)


Chunk 2: 100%|██████████| 25049/25049 [00:01<00:00, 22555.82it/s]


Processing chunk 3... (rows: 100000)


...

Processing chunk 119... (rows: 100000)


Chunk 119: 100%|██████████| 2151/2151 [00:00<00:00, 23853.64it/s]


Processing chunk 120... (rows: 55435)


Chunk 120: 100%|██████████| 1199/1199 [00:00<00:00, 23389.91it/s]



Step 1 Complete:
  Movies processed: 444,051
  Movies kept after filtering: 444,051
  Movies filtered out: 0
  Genres found: 26
  HAS_GENRE triples: 689,724


In [ ]:
# Step 2: Extract relations from title.crew.tsv
# Relations: (person) --DIRECTED--> (movie), (person) --WROTE--> (movie)
# If filtered mode: only process movies in kept_movies set

print("=" * 60)
print("Step 2: Processing title.crew.tsv")
print("=" * 60)

persons = set()
crew_path = get_file_path(FILES['crew'])
print(f"Reading: {crew_path}")
if USE_FILTERED_MODE:
    print(f"Filtering: only movies in kept set ({len(kept_movies):,} movies)")

chunks = read_tsv_chunked(crew_path, limit=LIMIT_ROWS)
directed_count = 0
wrote_count = 0
crew_rows_processed = 0
crew_rows_filtered = 0

for chunk_idx, chunk in enumerate(chunks):
    print(f"Processing chunk {chunk_idx + 1}... (rows: {len(chunk)})")

    # Filter out rows with missing tconst
    chunk = chunk[chunk['tconst'].notna() & (chunk['tconst'] != '\\N')]
    crew_rows_processed += len(chunk)

    if USE_FILTERED_MODE:
        # Filter to only keep movies that passed the movie filter
        chunk['tconst_str'] = chunk['tconst'].astype(str).str.strip()
        chunk = chunk[chunk['tconst_str'].isin(kept_movies)]
        crew_rows_filtered += len(chunk)

    for _, row in tqdm(chunk.iterrows(), total=len(chunk), desc=f"Chunk {chunk_idx + 1}"):
        movie_id = str(row['tconst']).strip()
        if not movie_id or movie_id == '\\N':
            continue

        # Double-check if in kept_movies (for filtered mode)
        if USE_FILTERED_MODE and movie_id not in kept_movies:
            continue

        # Extract directors
        directors = safe_split(row.get('directors', ''))
        for director_id in directors:
            if director_id:
                persons.add(director_id)
                triples.append(('DIRECTED', director_id, movie_id))
                directed_count += 1

        # Extract writers
        writers = safe_split(row.get('writers', ''))
        for writer_id in writers:
            if writer_id:
                persons.add(writer_id)
                triples.append(('WROTE', writer_id, movie_id))
                wrote_count += 1

print(f"\nStep 2 Complete:")
if USE_FILTERED_MODE:
    print(f"  Crew rows processed: {crew_rows_processed:,}")
    print(f"  Crew rows kept: {crew_rows_filtered:,}")
    print(f"  Crew rows filtered out: {crew_rows_processed - crew_rows_filtered:,}")
print(f"  Persons found so far: {len(persons):,}")
print(f"  DIRECTED triples: {directed_count:,}")
print(f"  WROTE triples: {wrote_count:,}")


Step 2: Processing title.crew.tsv
Reading: /content/drive/MyDrive/Knowledge Graph/title.crew.tsv
Filtering: only movies in kept set (444,051 movies)
Processing chunk 1... (rows: 100000)


Chunk 1: 100%|██████████| 22597/22597 [00:01<00:00, 20696.96it/s]


Processing chunk 2... (rows: 100000)


Chunk 2: 100%|██████████| 25049/25049 [00:01<00:00, 20935.32it/s]


Processing chunk 3... (rows: 100000)


...

Processing chunk 119... (rows: 100000)


Chunk 119: 100%|██████████| 2151/2151 [00:00<00:00, 21165.41it/s]


Processing chunk 120... (rows: 55435)


Chunk 120: 100%|██████████| 1199/1199 [00:00<00:00, 20532.28it/s]


Step 2 Complete:
  Crew rows processed: 11,955,435
  Crew rows kept: 444,051
  Crew rows filtered out: 11,511,384
  Persons found so far: 373,183
  DIRECTED triples: 493,393
  WROTE triples: 586,518


In [ ]:
# Step 3: Extract relations from title.principals.tsv
# Relations: (person) --ACTED_IN--> (movie) where category in {"actor", "actress"}
# If filtered mode: only process movies in kept_movies set AND ordering <= TOP_BILLING

print("=" * 60)
print("Step 3: Processing title.principals.tsv")
print("=" * 60)

principals_path = get_file_path(FILES['principals'])
print(f"Reading: {principals_path}")
if USE_FILTERED_MODE:
    print(f"Filtering: only movies in kept set, ordering <= {TOP_BILLING}")

chunks = read_tsv_chunked(principals_path, limit=LIMIT_ROWS)
acted_count = 0
principals_rows_processed = 0
principals_rows_filtered = 0

# Valid acting categories
ACTING_CATEGORIES = {'actor', 'actress'}

for chunk_idx, chunk in enumerate(chunks):
    print(f"Processing chunk {chunk_idx + 1}... (rows: {len(chunk)})")

    # Filter for acting roles only
    chunk = chunk[
        chunk['tconst'].notna() &
        (chunk['tconst'] != '\\N') &
        chunk['nconst'].notna() &
        (chunk['nconst'] != '\\N') &
        chunk['category'].notna()
    ]

    # Filter by category
    chunk = chunk[chunk['category'].str.lower().isin(ACTING_CATEGORIES)]
    principals_rows_processed += len(chunk)

    if USE_FILTERED_MODE:
        # Filter to only keep movies that passed the movie filter
        chunk['tconst_str'] = chunk['tconst'].astype(str).str.strip()
        chunk = chunk[chunk['tconst_str'].isin(kept_movies)]

        # Filter by ordering <= TOP_BILLING
        # Convert ordering to numeric, handling missing values
        chunk['ordering_numeric'] = pd.to_numeric(chunk['ordering'], errors='coerce')
        chunk = chunk[chunk['ordering_numeric'].notna() & (chunk['ordering_numeric'] <= TOP_BILLING)]
        principals_rows_filtered += len(chunk)

    for _, row in tqdm(chunk.iterrows(), total=len(chunk), desc=f"Chunk {chunk_idx + 1}"):
        movie_id = str(row['tconst']).strip()
        person_id = str(row['nconst']).strip()

        if movie_id and person_id and movie_id != '\\N' and person_id != '\\N':
            # Double-check if in kept_movies (for filtered mode)
            if USE_FILTERED_MODE and movie_id not in kept_movies:
                continue

            # Double-check ordering for filtered mode
            if USE_FILTERED_MODE:
                try:
                    ordering_val = pd.to_numeric(row.get('ordering'), errors='coerce')
                    if pd.isna(ordering_val) or ordering_val > TOP_BILLING:
                        continue
                except:
                    continue

            persons.add(person_id)
            triples.append(('ACTED_IN', person_id, movie_id))
            acted_count += 1

print(f"\nStep 3 Complete:")
if USE_FILTERED_MODE:
    print(f"  Principals rows processed: {principals_rows_processed:,}")
    print(f"  Principals rows kept: {principals_rows_filtered:,}")
    print(f"  Principals rows filtered out: {principals_rows_processed - principals_rows_filtered:,}")
print(f"  Total persons: {len(persons):,}")
print(f"  ACTED_IN triples: {acted_count:,}")


Step 3: Processing title.principals.tsv
Reading: /content/drive/MyDrive/Knowledge Graph/title.principals.tsv
Filtering: only movies in kept set, ordering <= 5
Processing chunk 1... (rows: 100000)


Chunk 1: 0it [00:00, ?it/s]


Processing chunk 2... (rows: 100000)


Chunk 2: 100%|██████████| 10/10 [00:00<00:00, 9236.52it/s]


Processing chunk 3... (rows: 100000)


...

Processing chunk 951... (rows: 100000)


Chunk 951: 100%|██████████| 806/806 [00:00<00:00, 20006.92it/s]


Processing chunk 952... (rows: 12527)


Chunk 952: 100%|██████████| 122/122 [00:00<00:00, 18173.28it/s]


Step 3 Complete:
  Principals rows processed: 39,624,377
  Principals rows kept: 1,493,177
  Principals rows filtered out: 38,131,200
  Total persons: 957,242
  ACTED_IN triples: 1,493,177


In [ ]:
# Step 4: Create entity and relation mappings
# Map string entities/relations to integer IDs

print("=" * 60)
print("Step 4: Creating entity and relation mappings")
print("=" * 60)

# Collect all entities
all_entities = set()
all_entities.update(movies)
all_entities.update(persons)
all_entities.update(genres)

# Create entity mapping (string -> integer ID)
entity_map = {entity: idx for idx, entity in enumerate(sorted(all_entities), start=1)}
# Reserve 0 for padding if needed (optional)

# Create relation mapping
relations = {'HAS_GENRE', 'DIRECTED', 'WROTE', 'ACTED_IN'}
relation_map = {rel: idx for idx, rel in enumerate(sorted(relations), start=1)}

print(f"Entity mapping created: {len(entity_map)} entities")
print(f"Relation mapping created: {len(relation_map)} relations")
print(f"\nRelations: {sorted(relations)}")


Step 4: Creating entity and relation mappings
Entity mapping created: 1401319 entities
Relation mapping created: 4 relations

Relations: ['ACTED_IN', 'DIRECTED', 'HAS_GENRE', 'WROTE']


In [ ]:
# Step 5: Convert triples to integer IDs and create final dataset

print("=" * 60)
print("Step 5: Converting triples to integer IDs")
print("=" * 60)

# Filter triples to only include entities that exist in our entity map
# (should be all, but safety check)
valid_triples = []
missing_entities = set()

for rel, head, tail in tqdm(triples, desc="Converting triples"):
    if head in entity_map and tail in entity_map and rel in relation_map:
        valid_triples.append({
            'head': entity_map[head],
            'relation': relation_map[rel],
            'tail': entity_map[tail],
            'head_str': head,
            'relation_str': rel,
            'tail_str': tail
        })
    else:
        if head not in entity_map:
            missing_entities.add(head)
        if tail not in entity_map:
            missing_entities.add(tail)

if missing_entities:
    print(f"Warning: {len(missing_entities)} entities not found in mapping (should not happen)")

print(f"Valid triples: {len(valid_triples)}")
print(f"Original triples: {len(triples)}")
print(f"Filtered out: {len(triples) - len(valid_triples)}")


Step 5: Converting triples to integer IDs


Converting triples: 100%|██████████| 3262812/3262812 [00:05<00:00, 628349.28it/s]

Valid triples: 3262812
Original triples: 3262812
Filtered out: 0


In [ ]:
# Step 6: Save outputs to Google Drive

print("=" * 60)
print("Step 6: Saving outputs")
print("=" * 60)

# Determine output filenames based on mode
if USE_FILTERED_MODE:
    triples_filename = 'triples_filtered.csv'
    entity_map_filename = 'entity_map_filtered.csv'
    relation_map_filename = 'relation_map_filtered.csv'
else:
    triples_filename = 'triples.csv'
    entity_map_filename = 'entity_map.csv'
    relation_map_filename = 'relation_map.csv'

# Save triples.csv (with integer IDs)
triples_df = pd.DataFrame(valid_triples)
triples_df[['head', 'relation', 'tail']].to_csv(
    os.path.join(OUTPUT_DIR, triples_filename),
    index=False
)
print(f"Saved: {OUTPUT_DIR}/{triples_filename}")
print(f"  Shape: {triples_df.shape}")

# Save entity_map.csv
entity_map_df = pd.DataFrame([
    {'entity_id': idx, 'entity': entity}
    for entity, idx in sorted(entity_map.items(), key=lambda x: x[1])
])
entity_map_df.to_csv(
    os.path.join(OUTPUT_DIR, entity_map_filename),
    index=False
)
print(f"Saved: {OUTPUT_DIR}/{entity_map_filename}")
print(f"  Shape: {entity_map_df.shape}")

# Save relation_map.csv
relation_map_df = pd.DataFrame([
    {'relation_id': idx, 'relation': rel}
    for rel, idx in sorted(relation_map.items(), key=lambda x: x[1])
])
relation_map_df.to_csv(
    os.path.join(OUTPUT_DIR, relation_map_filename),
    index=False
)
print(f"Saved: {OUTPUT_DIR}/{relation_map_filename}")
print(f"  Shape: {relation_map_df.shape}")

print("\nAll outputs saved successfully!")


Step 6: Saving outputs
Saved: /content/drive/MyDrive/kg_output_filtered/triples_filtered.csv
  Shape: (3262812, 6)
Saved: /content/drive/MyDrive/kg_output_filtered/entity_map_filtered.csv
  Shape: (1401319, 2)
Saved: /content/drive/MyDrive/kg_output_filtered/relation_map_filtered.csv
  Shape: (4, 2)

All outputs saved successfully!


In [ ]:
# Step 7: Print statistics

print("=" * 60)
print("KNOWLEDGE GRAPH STATISTICS")
print("=" * 60)

print(f"\nMode: {'FILTERED KG' if USE_FILTERED_MODE else 'FULL KG'}")

print(f"\nEntities:")
print(f"  Movies: {len(movies):,}")
print(f"  Persons: {len(persons):,}")
print(f"  Genres: {len(genres)}")
print(f"  Total entities: {len(all_entities):,}")

print(f"\nRelations:")
for rel in sorted(relations):
    count = sum(1 for t in valid_triples if t['relation_str'] == rel)
    print(f"  {rel}: {count:,} triples")

print(f"\nTotal triples: {len(valid_triples):,}")

# Comparison with full KG if in filtered mode
if USE_FILTERED_MODE and FULL_KG_STATS.get('triples'):
    reduction_ratio = len(valid_triples) / FULL_KG_STATS['triples']
    print(f"\nReduction vs Full KG:")
    print(f"  Full KG triples: {FULL_KG_STATS['triples']:,}")
    print(f"  Filtered KG triples: {len(valid_triples):,}")
    print(f"  Reduction ratio: {reduction_ratio:.2%}")
    print(f"  Size reduction: {(1 - reduction_ratio):.2%}")

if USE_FILTERED_MODE and FULL_KG_STATS.get('movies'):
    movie_reduction = len(movies) / FULL_KG_STATS['movies']
    print(f"\n  Full KG movies: {FULL_KG_STATS['movies']:,}")
    print(f"  Filtered KG movies: {len(movies):,}")
    print(f"  Movie reduction ratio: {movie_reduction:.2%}")

print(f"\nOutput files:")
if USE_FILTERED_MODE:
    print(f"  {OUTPUT_DIR}/triples_filtered.csv")
    print(f"  {OUTPUT_DIR}/entity_map_filtered.csv")
    print(f"  {OUTPUT_DIR}/relation_map_filtered.csv")
else:
    print(f"  {OUTPUT_DIR}/triples.csv")
    print(f"  {OUTPUT_DIR}/entity_map.csv")
    print(f"  {OUTPUT_DIR}/relation_map.csv")

print("\n" + "=" * 60)
print("Pipeline complete!")
print("=" * 60)


KNOWLEDGE GRAPH STATISTICS

Mode: FILTERED KG

Entities:
  Movies: 444,051
  Persons: 957,242
  Genres: 26
  Total entities: 1,401,319

Relations:
  ACTED_IN: 1,493,177 triples
  DIRECTED: 493,393 triples
  HAS_GENRE: 689,724 triples
  WROTE: 586,518 triples

Total triples: 3,262,812

Reduction vs Full KG:
  Full KG triples: 81,000,000
  Filtered KG triples: 3,262,812
  Reduction ratio: 4.03%
  Size reduction: 95.97%

  Full KG movies: 16,500,000
  Filtered KG movies: 444,051
  Movie reduction ratio: 2.69%

Output files:
  /content/drive/MyDrive/kg_output_filtered/triples_filtered.csv
  /content/drive/MyDrive/kg_output_filtered/entity_map_filtered.csv
  /content/drive/MyDrive/kg_output_filtered/relation_map_filtered.csv

Pipeline complete!


# TransE Training Stage with PyKEEN

This section trains TransE embeddings on the filtered Knowledge Graph.

**Steps:**
1. Load filtered triples from Google Drive
2. Create PyKEEN TriplesFactory
3. Train/validation/test split
4. Train TransE model
5. Evaluate with ranking metrics
6. Save embeddings and metadata

In [1]:
# Install PyKEEN and dependencies
%pip install pykeen -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.5 MB/s eta 0:00:00


In [2]:
# TransE Training Configuration
import pandas as pd
import numpy as np
import random
import json
import torch
from pathlib import Path
from pykeen.triples import TriplesFactory
from pykeen.evaluation import RankBasedEvaluator
from pykeen.pipeline import pipeline
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

# ============================================================================
# TRANSE TRAINING CONFIGURATION
# ============================================================================
EMBEDDING_DIM = 128
NUM_EPOCHS = 30
BATCH_SIZE = 512  # Reasonable for Colab GPU
LEARNING_RATE = 0.001
MARGIN = 1.0  # Margin for MarginRankingLoss
NEGATIVE_SAMPLER_KWARGS = {'num_negs_per_pos': 1}  # Default negative sampling

# Train/validation/test split ratios
TRAIN_RATIO = 0.8
VALID_RATIO = 0.1
TEST_RATIO = 0.1

# Paths
FILTERED_KG_DIR = '/content/drive/MyDrive/kg_output_filtered'
TRIPLES_FILE = f'{FILTERED_KG_DIR}/triples_filtered.csv'
ENTITY_MAP_FILE = f'{FILTERED_KG_DIR}/entity_map_filtered.csv'
RELATION_MAP_FILE = f'{FILTERED_KG_DIR}/relation_map_filtered.csv'
OUTPUT_DIR = f'{FILTERED_KG_DIR}/pykeen_transe'

# Dry-run mode: limit triples for quick testing
DRY_RUN_LIMIT = None  # Set to integer (e.g., 10000) for quick testing

# Device configuration
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print("=" * 60)
print("TRANSE TRAINING CONFIGURATION")
print("=" * 60)
print(f"Embedding dimension: {EMBEDDING_DIM}")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Margin: {MARGIN}")
print(f"Device: {DEVICE}")
print(f"Train/Valid/Test split: {TRAIN_RATIO:.0%}/{VALID_RATIO:.0%}/{TEST_RATIO:.0%}")
print(f"Dry-run limit: {DRY_RUN_LIMIT if DRY_RUN_LIMIT else 'None (full dataset)'}")
print("=" * 60)

INFO:pykeen.utils:Using opt_einsum


TRANSE TRAINING CONFIGURATION
Embedding dimension: 128
Epochs: 30
Batch size: 512
Learning rate: 0.001
Margin: 1.0
Device: cuda
Train/Valid/Test split: 80%/10%/10%
Dry-run limit: None (full dataset)


In [3]:
# Mount Google Drive (if not already mounted)
from google.colab import drive
try:
    drive.mount('/content/drive', force_remount=False)
except:
    print("Drive already mounted or not in Colab environment")

# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
print(f"Output directory ready: {OUTPUT_DIR}")

Mounted at /content/drive
Output directory ready: /content/drive/MyDrive/kg_output_filtered/pykeen_transe


In [4]:
# Helper function: Load triples from CSV
def load_triples(filepath, limit=None):
    """
    Load triples from CSV file.

    Args:
        filepath: Path to triples CSV file
        limit: Optional limit on number of rows to read (for dry-run)

    Returns:
        DataFrame with columns: head, relation, tail
    """
    print(f"Loading triples from: {filepath}")

    if limit:
        print(f"  Dry-run mode: limiting to {limit:,} rows")
        triples_df = pd.read_csv(filepath, nrows=limit)
    else:
        triples_df = pd.read_csv(filepath)

    print(f"  Loaded {len(triples_df):,} triples")
    print(f"  Columns: {list(triples_df.columns)}")

    # Ensure we have the expected columns
    required_cols = ['head', 'relation', 'tail']
    if not all(col in triples_df.columns for col in required_cols):
        raise ValueError(f"Expected columns {required_cols}, got {list(triples_df.columns)}")

    # Remove any rows with missing values
    initial_count = len(triples_df)
    triples_df = triples_df.dropna(subset=required_cols)
    if len(triples_df) < initial_count:
        print(f"  Removed {initial_count - len(triples_df):,} rows with missing values")

    # Convert to string and strip whitespace
    for col in required_cols:
        triples_df[col] = triples_df[col].astype(str).str.strip()

    # Remove empty strings
    triples_df = triples_df[
        (triples_df['head'] != '') &
        (triples_df['relation'] != '') &
        (triples_df['tail'] != '')
    ]

    print(f"  Final triples count: {len(triples_df):,}")
    return triples_df

# Load triples
triples_df = load_triples(TRIPLES_FILE, limit=DRY_RUN_LIMIT)

Loading triples from: /content/drive/MyDrive/kg_output_filtered/triples_filtered.csv
  Loaded 3,262,812 triples
  Columns: ['head', 'relation', 'tail']
  Final triples count: 3,262,812


In [5]:
# Helper function: Build TriplesFactory from labeled triples
def build_factory(triples_df, create_inverse_triples=False):
    """
    Create PyKEEN TriplesFactory from DataFrame with string labels.

    Args:
        triples_df: DataFrame with columns head, relation, tail (string labels)
        create_inverse_triples: Whether to create inverse triples

    Returns:
        TriplesFactory object
    """
    print("=" * 60)
    print("Building TriplesFactory")
    print("=" * 60)

    # Convert to numpy array of string triples
    triples_array = triples_df[['head', 'relation', 'tail']].values

    print(f"  Input triples shape: {triples_array.shape}")
    print(f"  Unique entities: {len(set(triples_array[:, 0]) | set(triples_array[:, 2])):,}")
    print(f"  Unique relations: {len(set(triples_array[:, 1]))}")

    # Create TriplesFactory from labeled triples
    factory = TriplesFactory.from_labeled_triples(
        triples=triples_array,
        create_inverse_triples=create_inverse_triples
    )

    print(f"  Factory created successfully")
    print(f"  Number of entities: {factory.num_entities:,}")
    print(f"  Number of relations: {factory.num_relations}")
    print(f"  Number of triples: {factory.num_triples:,}")

    return factory

# Build TriplesFactory
triples_factory = build_factory(triples_df, create_inverse_triples=False)

Building TriplesFactory
  Input triples shape: (3262812, 3)
  Unique entities: 1,401,319
  Unique relations: 4
  Factory created successfully
  Number of entities: 1,401,319
  Number of relations: 4
  Number of triples: 3,238,794


In [6]:
# Create train/validation/test split
print("=" * 60)
print("Creating train/validation/test split")
print("=" * 60)

training, validation, testing = triples_factory.split(
    ratios=[TRAIN_RATIO, VALID_RATIO, TEST_RATIO],
    random_state=RANDOM_SEED
)

print(f"Training triples: {training.num_triples:,} ({training.num_triples/triples_factory.num_triples:.1%})")
print(f"Validation triples: {validation.num_triples:,} ({validation.num_triples/triples_factory.num_triples:.1%})")
print(f"Test triples: {testing.num_triples:,} ({testing.num_triples/triples_factory.num_triples:.1%})")
print(f"Random seed: {RANDOM_SEED}")

Creating train/validation/test split


INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [1189743, 323879, 323880]


Training triples: 2,591,035 (80.0%)
Validation triples: 323,879 (10.0%)
Test triples: 323,880 (10.0%)
Random seed: 42


In [9]:
# Helper function: Train TransE model
def train_model(training, validation, testing, device='cuda'):
    """
    Train TransE model using PyKEEN pipeline.

    Args:
        training: Training TriplesFactory
        validation: Validation TriplesFactory
        testing: Testing TriplesFactory
        device: Device to use ('cuda' or 'cpu')

    Returns:
        Trained model and training results
    """
    print("=" * 60)
    print("Training TransE Model")
    print("=" * 60)
    print(f"Device: {device}")
    print(f"Embedding dimension: {EMBEDDING_DIM}")
    print(f"Epochs: {NUM_EPOCHS}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Learning rate: {LEARNING_RATE}")
    print("=" * 60)

    # Use PyKEEN pipeline for training
    # PyKEEN requires both training and testing to be specified
    result = pipeline(
        training=training,
        validation=validation,
        testing=testing,
        model='TransE',
        model_kwargs=dict(
            embedding_dim=EMBEDDING_DIM,
        ),
        training_kwargs=dict(
            num_epochs=NUM_EPOCHS,
            batch_size=BATCH_SIZE,
            use_tqdm=True,
        ),
        training_loop='SLCWA',
        negative_sampler='basic',
        negative_sampler_kwargs=NEGATIVE_SAMPLER_KWARGS,
        loss='marginranking',
        loss_kwargs=dict(margin=MARGIN),
        optimizer='Adam',
        optimizer_kwargs={'lr': LEARNING_RATE},
        device=device,
        random_seed=RANDOM_SEED,
    )

    print("\nTraining completed!")
    return result

# Train the model
training_result = train_model(training, validation, testing, device=DEVICE)
model = training_result.model

INFO:pykeen.pipeline.api:Using device: cuda
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()


Training TransE Model
Device: cuda
Embedding dimension: 128
Epochs: 30
Batch size: 512
Learning rate: 0.001


INFO:pykeen.nn.representation:Inferred unique=False for Embedding()


Training epochs on cuda:0:   0%|          | 0/30 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/5.06k [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/5.06k [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/5.06k [00:00<?, ?batch/s]

Evaluating on cuda:0:   0%|          | 0.00/324k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 6739.09s seconds



Training completed!


In [26]:
# Fast evaluation: evaluate on subset of test triples for faster runtime
FAST_EVAL = True  # Set to True to evaluate on random subset (20,000 triples)
FAST_EVAL_SIZE = 20000  # Number of triples to use for fast evaluation

# Helper function: Evaluate model with ranking metrics
def evaluate_model(model, testing, training, validation, device='cuda'):
    """
    Evaluate model on test set using ranking metrics.

    Args:
        model: Trained PyKEEN model
        testing: Test TriplesFactory
        training: Training TriplesFactory (for filtering)
        validation: Validation TriplesFactory (for filtering)
        device: Device to use

    Returns:
        Dictionary of evaluation metrics
    """
    print("=" * 60)
    print("Evaluating Model on Test Set")
    print("=" * 60)

    evaluator = RankBasedEvaluator(metrics=["hits@k", "mrr", "mr"])

    # Move model to device
    model = model.to(device)

    # Prepare test triples
    test_triples = testing.mapped_triples

    # Fast evaluation: use random subset if enabled
    if FAST_EVAL and len(test_triples) > FAST_EVAL_SIZE:
        print(f"Fast evaluation enabled: sampling {FAST_EVAL_SIZE:,} triples from {len(test_triples):,} test triples")
        # Set random seed for reproducibility
        generator = torch.Generator(device=device)
        generator.manual_seed(RANDOM_SEED)

        # Randomly sample indices
        indices = torch.randperm(len(test_triples))[:FAST_EVAL_SIZE]
        test_triples = test_triples[indices]
        print(f"  Using {len(test_triples):,} triples for evaluation")
    else:
        print(f"Evaluating on full test set: {len(test_triples):,} triples")

    # Prepare additional filter triples (training and validation sets)
    # These are used to filter out known triples during evaluation
    additional_filter_triples = [
        training.mapped_triples,
        validation.mapped_triples
    ]

    # Evaluate
    metrics = evaluator.evaluate(
        model=model,
        mapped_triples=test_triples,
        additional_filter_triples=additional_filter_triples,
        batch_size=BATCH_SIZE,
        device=device,
    )

    # Convert RankBasedMetricResults to dictionary
    metrics_dict = metrics.to_dict()

    print("\nRanking Metrics:")
    print(f"  Hits@1: {metrics_dict.get('hits_at_1', 0):.4f}")
    print(f"  Hits@3: {metrics_dict.get('hits_at_3', 0):.4f}")
    print(f"  Hits@10: {metrics_dict.get('hits_at_10', 0):.4f}")
    print(f"  Mean Rank: {metrics_dict.get('mean_rank', 0):.2f}")
    print(f"  Mean Reciprocal Rank: {metrics_dict.get('mean_reciprocal_rank', 0):.4f}")

    return metrics_dict

# Evaluate model
test_metrics = evaluate_model(model, testing, training, validation, device=DEVICE)

Evaluating Model on Test Set
Fast evaluation enabled: sampling 20,000 triples from 323,880 test triples
  Using 20,000 triples for evaluation


Evaluating on cuda:0:   0%|          | 0.00/20.0k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 428.41s seconds



Ranking Metrics:
  Hits@1: 0.0000
  Hits@3: 0.0000
  Hits@10: 0.0000
  Mean Rank: 0.00
  Mean Reciprocal Rank: 0.0000


In [28]:
m = test_metrics["both"]["realistic"]

print("Hits@1:", m["hits_at_1"])
print("Hits@3:", m["hits_at_3"])
print("Hits@10:", m["hits_at_10"])

# نزدیک‌ترین معادل MR/MRR در این خروجی‌ها:
print("Mean Rank (arithmetic):", m["arithmetic_mean_rank"])
print("Harmonic Mean Rank:", m["harmonic_mean_rank"])
print("Inverse Harmonic Mean Rank:", m["inverse_harmonic_mean_rank"])


Hits@1: 0.00055
Hits@3: 0.061625
Hits@10: 0.087425
Mean Rank (arithmetic): 173133.46875
Harmonic Mean Rank: 27.957876205444336
Inverse Harmonic Mean Rank: 0.03576809540390968


In [32]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

5692

In [33]:
# Helper function: Compute Precision@K, Recall@K, F1@K
def compute_precision_recall_f1(model, testing, k=10, device='cuda', batch_size=512):
    """
    Compute Precision@K, Recall@K, and F1@K for test triples.

    Args:
        model: Trained PyKEEN model
        testing: Test TriplesFactory
        k: Top-K for precision/recall
        device: Device to use
        batch_size: Batch size for evaluation

    Returns:
        Dictionary with precision@k, recall@k, f1@k
    """
    print(f"\nComputing Precision@{k}, Recall@{k}, F1@{k}...")

    model = model.to(device)
    model.eval()

    test_triples = testing.mapped_triples.to(device)
    num_triples = len(test_triples)

    true_positives = 0
    total_positive = num_triples

    # Process in batches
    with torch.no_grad():
        for i in range(0, num_triples, batch_size):
            batch = test_triples[i:i+batch_size]

            # For each triple, get top-k predictions for tail
            h, r, t = batch[:, 0], batch[:, 1], batch[:, 2]

            # Get scores for all possible tails
            scores = model.score_t(batch)

            # Get top-k predictions
            _, top_k_indices = torch.topk(scores, k=k, dim=1)

            # Check if true tail is in top-k
            true_tails = t.unsqueeze(1)
            matches = (top_k_indices == true_tails).any(dim=1)
            true_positives += matches.sum().item()

    precision_k = true_positives / (num_triples * k) if num_triples > 0 else 0.0
    recall_k = true_positives / total_positive if total_positive > 0 else 0.0
    f1_k = 2 * (precision_k * recall_k) / (precision_k + recall_k) if (precision_k + recall_k) > 0 else 0.0

    return {
        f'precision_at_{k}': precision_k,
        f'recall_at_{k}': recall_k,
        f'f1_at_{k}': f1_k
    }

# Compute Precision@10, Recall@10, F1@10
prf_metrics = compute_precision_recall_f1(model, testing, k=10, device=DEVICE, batch_size=BATCH_SIZE)
print(f"  Precision@10: {prf_metrics['precision_at_10']:.4f}")
print(f"  Recall@10: {prf_metrics['recall_at_10']:.4f}")
print(f"  F1@10: {prf_metrics['f1_at_10']:.4f}")

# Merge with test_metrics
test_metrics.update(prf_metrics)


Computing Precision@10, Recall@10, F1@10...


OutOfMemoryError: CUDA out of memory. Tried to allocate 342.12 GiB. GPU 0 has a total capacity of 14.74 GiB of which 2.59 GiB is free. Process 14820 has 12.15 GiB memory in use. Of the allocated memory 2.02 GiB is allocated by PyTorch, and 10.01 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [36]:
# Helper function: Save embeddings and metadata
def save_artifacts(model, triples_factory, training, validation, testing, test_metrics, output_dir):
    """
    Save model embeddings, mappings, and metadata to disk.

    Args:
        model: Trained PyKEEN model
        triples_factory: TriplesFactory used for training
        training_result: Training result object
        testing: Test TriplesFactory
        test_metrics: Test set evaluation metrics
        output_dir: Output directory path
    """
    print("=" * 60)
    print("Saving Artifacts")
    print("=" * 60)

    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    # Extract embeddings
    entity_embeddings = model.entity_representations[0](indices=None).detach().cpu().numpy()
    relation_embeddings = model.relation_representations[0](indices=None).detach().cpu().numpy()

    print(f"Entity embeddings shape: {entity_embeddings.shape}")
    print(f"Relation embeddings shape: {relation_embeddings.shape}")

    # Save embeddings as numpy arrays
    np.save(output_path / 'entity_embeddings.npy', entity_embeddings)
    print(f"  Saved: {output_path / 'entity_embeddings.npy'}")

    np.save(output_path / 'relation_embeddings.npy', relation_embeddings)
    print(f"  Saved: {output_path / 'relation_embeddings.npy'}")

    # Save entity and relation mappings
    entity_to_id = {entity: int(idx) for entity, idx in triples_factory.entity_to_id.items()}
    relation_to_id = {relation: int(idx) for relation, idx in triples_factory.relation_to_id.items()}

    with open(output_path / 'entity_to_id.json', 'w') as f:
        json.dump(entity_to_id, f, indent=2)
    print(f"  Saved: {output_path / 'entity_to_id.json'}")

    with open(output_path / 'relation_to_id.json', 'w') as f:
        json.dump(relation_to_id, f, indent=2)
    print(f"  Saved: {output_path / 'relation_to_id.json'}")

    # Create metadata dictionary
    metadata = {
        'model': 'TransE',
        'embedding_dim': EMBEDDING_DIM,
        'num_epochs': NUM_EPOCHS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'margin': MARGIN,
        'random_seed': RANDOM_SEED,
        'device': DEVICE,
        'dataset_sizes': {
            'total_triples': int(triples_factory.num_triples),
            'train_triples': int(training.num_triples),
            'valid_triples': int(validation.num_triples),
            'test_triples': int(testing.num_triples),
        },
        'num_entities': int(triples_factory.num_entities),
        'num_relations': int(triples_factory.num_relations),
        'evaluation_metrics': {k: float(v) if isinstance(v, (int, float, np.number)) else str(v)
                               for k, v in test_metrics.items()},
        'training_loss': float(training_result.losses[-1]) if hasattr(training_result, 'losses') and training_result.losses else None,
    }

    # Save metadata
    with open(output_path / 'metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"  Saved: {output_path / 'metadata.json'}")

    # Try to save model checkpoint if available
    try:
        checkpoint_path = output_path / 'model_checkpoint.pt'
        torch.save({
            'model_state_dict': model.state_dict(),
            'model_kwargs': {'embedding_dim': EMBEDDING_DIM},
            'entity_to_id': entity_to_id,
            'relation_to_id': relation_to_id,
        }, checkpoint_path)
        print(f"  Saved: {checkpoint_path}")
    except Exception as e:
        print(f"  Warning: Could not save model checkpoint: {e}")

    print("\nAll artifacts saved successfully!")
    return metadata

# Save all artifacts
metadata = save_artifacts(model, triples_factory, training, validation, testing, test_metrics, OUTPUT_DIR)

Saving Artifacts
Entity embeddings shape: (1401319, 128)
Relation embeddings shape: (4, 128)
  Saved: /content/drive/MyDrive/kg_output_filtered/pykeen_transe/entity_embeddings.npy
  Saved: /content/drive/MyDrive/kg_output_filtered/pykeen_transe/relation_embeddings.npy
  Saved: /content/drive/MyDrive/kg_output_filtered/pykeen_transe/entity_to_id.json
  Saved: /content/drive/MyDrive/kg_output_filtered/pykeen_transe/relation_to_id.json
  Saved: /content/drive/MyDrive/kg_output_filtered/pykeen_transe/metadata.json
  Saved: /content/drive/MyDrive/kg_output_filtered/pykeen_transe/model_checkpoint.pt

All artifacts saved successfully!


In [37]:
# Print final summary
print("=" * 60)
print("TRANSE TRAINING SUMMARY")
print("=" * 60)

print(f"\nModel Configuration:")
print(f"  Model: TransE")
print(f"  Embedding dimension: {EMBEDDING_DIM}")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: {LEARNING_RATE}")

print(f"\nDataset:")
print(f"  Total entities: {triples_factory.num_entities:,}")
print(f"  Total relations: {triples_factory.num_relations}")
print(f"  Total triples: {triples_factory.num_triples:,}")
print(f"  Train: {training.num_triples:,} ({training.num_triples/triples_factory.num_triples:.1%})")
print(f"  Valid: {validation.num_triples:,} ({validation.num_triples/triples_factory.num_triples:.1%})")
print(f"  Test: {testing.num_triples:,} ({testing.num_triples/triples_factory.num_triples:.1%})")

print(f"\nTest Set Evaluation:")
for metric_name, metric_value in test_metrics.items():
    if isinstance(metric_value, (int, float, np.number)):
        print(f"  {metric_name}: {metric_value:.4f}")

print(f"\nSaved Artifacts:")
print(f"  {OUTPUT_DIR}/entity_embeddings.npy")
print(f"  {OUTPUT_DIR}/relation_embeddings.npy")
print(f"  {OUTPUT_DIR}/entity_to_id.json")
print(f"  {OUTPUT_DIR}/relation_to_id.json")
print(f"  {OUTPUT_DIR}/metadata.json")

print("\n" + "=" * 60)
print("TransE Training Complete!")
print("=" * 60)

TRANSE TRAINING SUMMARY

Model Configuration:
  Model: TransE
  Embedding dimension: 128
  Epochs: 30
  Batch size: 512
  Learning rate: 0.001

Dataset:
  Total entities: 1,401,319
  Total relations: 4
  Total triples: 3,238,794
  Train: 2,591,035 (80.0%)
  Valid: 323,879 (10.0%)
  Test: 323,880 (10.0%)

Test Set Evaluation:

Saved Artifacts:
  /content/drive/MyDrive/kg_output_filtered/pykeen_transe/entity_embeddings.npy
  /content/drive/MyDrive/kg_output_filtered/pykeen_transe/relation_embeddings.npy
  /content/drive/MyDrive/kg_output_filtered/pykeen_transe/entity_to_id.json
  /content/drive/MyDrive/kg_output_filtered/pykeen_transe/relation_to_id.json
  /content/drive/MyDrive/kg_output_filtered/pykeen_transe/metadata.json

TransE Training Complete!
